In [3]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import coo_matrix
import torch.nn as nn
# import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print("Device:",device)


ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

In [ ]:
# Loading in user data [user_id,song_id,play_count,last_played,user_age,user_country]
interaction_data = pd.read_csv('data/user/user_data.csv')

# Creating mappings for users and items
user_mapping = {user: idx for idx, user in enumerate(interaction_data['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(interaction_data['song_id'].unique())}

interaction_data['user_id'] = interaction_data['user_id'].map(user_mapping)
interaction_data['song_id'] = interaction_data['song_id'].map(item_mapping)

# Create a sparse matrix of interactions
rows = interaction_data['user_id'].values
cols = interaction_data['song_id'].values
data = interaction_data['play_count'].values

interaction_matrix = coo_matrix((data, (rows, cols)), shape=(len(user_mapping), len(item_mapping)))

# Convert to PyTorch geometric data format
edge_index, edge_attr = from_scipy_sparse_matrix(interaction_matrix)



In [ ]:
# Creating the model
class GCNRecommender(nn.Module):
    def __init__(self, num_users, num_items, latent_dim):
        super(GCNRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)
        self.conv1 = GCNConv(latent_dim, 128)
        self.conv2 = GCNConv(128, 64)
        self.fc = nn.Linear(64, 1)

    def forward(self, user, item, edge_index):
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)
        x = torch.cat([user_embed, item_embed], dim=0)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = torch.cat([x[user], x[item]], dim=1)
        x = self.fc(x)
        return x
    


In [ ]:
# Create a Torch geometric dat object
data = Data(edge_index=edge_index, edge_attr=edge_attr).to(device)

# Definine model metrics (loss, optimization) and hyperparameters (num_users, num_items, latent_dim)
num_users = len(user_mapping)
num_items = len(item_mapping)
latent_dim = 16

model = GCNRecommender(num_users, num_items, latent_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [ ]:
# Defining model training function
def model_training(epochs):
    model.train()
    loss_list = {}
    for epoch in range(0,epochs +1):
        optimizer.zero_grad()
        output = model(data.edge_index, data.edge_attr)
        loss = criterion(output, data.edge_attr.float())
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        loss_list[epoch] = loss.item()
    
    return loss_list



In [ ]:
# Defining evaluation function
def model_evaluation(): 
    model.eval()
    with torch.no_grad():
        